<a href="https://colab.research.google.com/github/Shruti1102000/Text-Classification3-using-Entities-Logistic-Regression-and-Naive-Bayes/blob/main/Text_Classification_Entities_Logistic_Regression_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing necessory libraries

In [ ]:
from nltk.corpus import reuters
from sklearn.preprocessing import MultiLabelBinarizer
import spacy
import numpy as np

'''
  References:
  https://spacy.io/usage/vectors-similarity
  https://spacy.io/usage/linguistic-features#named-entities
  https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html
  https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html
  https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
'''

'\n  References:\n  https://spacy.io/usage/vectors-similarity\n  https://spacy.io/usage/linguistic-features#named-entities\n  https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html\n  https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html\n  https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html\n'

In [ ]:
import nltk
nltk.download('reuters')
!python -m spacy download en_core_web_lg

[nltk_data] Downloading package reuters to /root/nltk_data...
     |████████████████████████████████| 827.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180945 sha256=90972ed414e7d8bbece0fd333e5430d7bcf8482f604546dfdbc22902653afce0
  Stored in directory: /tmp/pip-ephem-wheel-cache-os0n1itv/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


# Loading Data 

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
mlb = MultiLabelBinarizer()

In [ ]:
def collection_stats():
  documents = reuters.fileids()
  print(str(len(documents)) + " documents");

  train_docs = list(filter(lambda doc: doc.startswith("train"), documents));
  print(str(len(train_docs)) + " total train documents");
 
  test_docs = list(filter(lambda doc: doc.startswith("test"), documents));
  print(str(len(test_docs)) + " total test documents")

  categories = reuters.categories()

  print(str(len(categories)) + " categories");

In [ ]:
collection_stats()

10788 documents
7769 total train documents
3019 total test documents
90 categories


# Train Test Split of data

In [ ]:
def train_test_split():
  documents = reuters.fileids()
  train_docs = [document for document in documents if document.startswith("train")]
  test_docs = [document for document in documents if document.startswith("test")]
  x_train = [reuters.raw(doc_id) for doc_id in train_docs]
  y_train = [reuters.raw(doc_id) for doc_id in test_docs]
  x_test = mlb.fit_transform([reuters.categories(doc_id) for doc_id in train_docs])
  y_test = mlb.transform([reuters.categories(doc_id) for doc_id in test_docs])
  return x_train, y_train, x_test, y_test

In [ ]:
x_train, x_test, y_train, y_test = train_test_split()

# Functions for getting entities and word vectors

In [ ]:
def get_entities(text):
  """ returns the entities of the input text """
  tokens = nlp(text)
  entities = [token.text for token in tokens.ents]
  return entities

In [ ]:
def get_word_vectors(sentence):
  """ returns the vector representation of the input sentence """
  tokens = nlp(sentence)
  vector = np.sum([token.vector for token in tokens], axis=0)
  if vector.shape == ():
    vector = np.random.rand(300,)
  return vector

## Build vector representations for train data

In [ ]:
# Extract entities and build vector representations for train data
x_train = [get_word_vectors(' '.join(get_entities(doc))) for doc in x_train]
print(np.shape(x_train))

(7769, 300)


## Build vector representations for test data

In [ ]:
# Extract entities and build vector representations for test data
x_test = [get_word_vectors(' '.join(get_entities(doc))) for doc in x_test]
print(np.shape(x_test))

(3019, 300)


# Naive bayes classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier

## Fit and predict using Naive bayes classifier

In [ ]:
gnb = OneVsRestClassifier(GaussianNB())
gnb.fit(x_train, y_train)

OneVsRestClassifier(estimator=GaussianNB(priors=None, var_smoothing=1e-09),
                    n_jobs=None)

In [ ]:
y_pred_nb = gnb.predict(x_test)

## Print classification report

In [ ]:
print("Naive Bayes Classifier Result")
print(classification_report(y_true=y_test, y_pred=y_pred_nb))

Naive Bayes Classifier Result
              precision    recall  f1-score   support

           0       0.34      0.89      0.49       719
           1       0.01      0.65      0.02        23
           2       0.05      0.36      0.09        14
           3       0.04      0.43      0.08        30
           4       0.04      0.83      0.08        18
           5       0.00      0.00      0.00         1
           6       0.05      0.78      0.10        18
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         3
           9       0.05      0.50      0.09        28
          10       0.02      0.61      0.04        18
          11       0.00      0.00      0.00         1
          12       0.05      0.34      0.08        56
          13       0.04      0.35      0.07        20
          14       0.00      0.00      0.00         2
          15       0.19      0.64      0.29        28
          16       0.00      0.00      0.00        

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

## Fit and predict using Logistic Regression Classifier 

In [ ]:
lr = OneVsRestClassifier(LogisticRegression(solver='newton-cg', n_jobs=4))
lr.fit(x_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto', n_jobs=4,
                                                 penalty='l2',
                                                 random_state=None,
                                                 solver='newton-cg', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [ ]:
y_pred = lr.predict(x_test)

## Print classification report

In [ ]:
print("Logistic Regression result")
print(classification_report(y_true=y_test, y_pred=y_pred))

Logistic Regression result
              precision    recall  f1-score   support

           0       0.87      0.79      0.83       719
           1       0.27      0.13      0.18        23
           2       0.35      0.43      0.39        14
           3       0.16      0.20      0.18        30
           4       0.18      0.28      0.22        18
           5       0.00      0.00      0.00         1
           6       0.40      0.44      0.42        18
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         3
           9       0.47      0.57      0.52        28
          10       0.26      0.33      0.29        18
          11       0.00      0.00      0.00         1
          12       0.36      0.52      0.43        56
          13       0.35      0.30      0.32        20
          14       0.00      0.00      0.00         2
          15       0.28      0.25      0.26        28
          16       0.00      0.00      0.00         1


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
